In [1]:
from config import *
import pandas as pd
import numpy as np
from main_package.bkt_pyKT import convert_df_strings_to_arrays, get_question_level_prediction
from main_package.utils import data_path_to_abs_path
from main_package.bkt_pyKT_per_skill import train_bkt, evaluate_bkt
import copy
from sklearn.metrics import roc_auc_score, accuracy_score

from pyBKT.models import Model
np.seterr(divide='ignore', invalid='ignore') #pyBKT raises some warnings that I should ignore

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [21]:
sample = False

data_folder_path_major_skill = 'isaac/pyKT_processed/sample/' if sample else 'isaac/pyKT_processed/'
data_folder_path_first_skill = 'isaac/pyKT_processed/first_skill/sample/' if sample else 'isaac/pyKT_processed/first_skill/'

data_folder_path = data_folder_path_major_skill

df_train = pd.read_csv(data_path_to_abs_path(data_folder_path + 'train_valid.csv'))
convert_df_strings_to_arrays(df_train)
df_test = pd.read_csv(data_path_to_abs_path(data_folder_path + 'test.csv'))
convert_df_strings_to_arrays(df_test)

In [22]:
def truncate(df: pd.DataFrame, max_interaction_len=200):
    for column_name in ['questions', 'concepts', 'responses', 'is_repeat']:
        if column_name in df.columns:
            df[column_name] = df[column_name].apply(lambda x: x[:max_interaction_len])
    return df

def transform_pyKT_to_simple_format(df: pd.DataFrame):
    df = copy.deepcopy(df)
    df['user_ids'] = df.apply(
        lambda row: np.full_like(row['questions'], row['uid']),
        axis=1,
    )
    d = {
        column_name: np.concatenate(df[column_name + 's']) for column_name in ['question', 'concept', 'response', 'user_id'] 
    }
    d['is_repeat'] = np.concatenate(df['is_repeat'])
    df = pd.DataFrame(data=d)
    return df

In [23]:
df_train = transform_pyKT_to_simple_format(truncate(df_train))
df_test = transform_pyKT_to_simple_format(truncate(df_test))

In [24]:
defaults = {'skill_name': 'concept', 'correct': 'response'}
model = Model(seed = 42, num_fits = 2, defaults=defaults)

In [25]:
model.fit(data = df_train, forgets=True)
print(model.params())

                        value
skill param   class          
0     prior   default 0.27594
      learns  default 0.11392
      guesses default 0.21132
      slips   default 0.33667
      forgets default 0.40095
...                       ...
93    prior   default 0.92926
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000

[470 rows x 1 columns]


In [26]:
predictions = model.predict(data = df_test)

In [27]:
predictions # I should use correct_predictions (that is what they use in eval)

,question,concept,response,user_id,is_repeat,correct_predictions,state_predictions
46343,6,7,1,1,0,0.47821,0.41228
46344,6,11,1,1,1,0.45728,0.48004
46345,6,12,1,1,1,0.47451,0.36828
46346,7,7,0,1,0,0.50596,0.47841
46347,7,8,0,1,1,0.40993,0.34311
...,...,...,...,...,...,...,...
1836,223,12,0,999,1,0.47451,0.36828
1837,223,11,0,999,1,0.45728,0.48004
1838,223,10,0,999,1,0.30497,0.10573
1839,223,9,0,999,1,0.37100,0.23616


In [28]:
df_results = predictions.groupby('user_id').agg({
    column_name: list for column_name in ['response', 'correct_predictions', 'is_repeat']
})
df_results = df_results.reset_index(drop=True).rename(columns={
    'response': 'responses',
    'correct_predictions': 'predictions',
})
df_results

,responses,predictions,is_repeat
0,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0.4782124735864324, 0.45728017590238107, 0.47...","[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, ..."
1,"[1, 1, 1, 1, 1, 1, 1]","[0.3003772569944446, 0.45430000541371296, 0.47...","[0, 1, 0, 0, 0, 1, 1]"
2,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0.4782124735864324, 0.420924907423684, 0.4745...","[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0.5125773143200658, 0.3937033345643633, 0.474...","[0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, ..."
4,"[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0.3709954701010898, 0.40649335060004954, 0.33...","[0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."
...,...,...,...
888,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.45728017590238107, 0.4349981234409045, 0.47...","[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ..."
889,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, ...","[0.3709954701010898, 0.40649335060004954, 0.40...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, ..."
890,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, ...","[0.3178868020830998, 0.33604595706832907, 0.28...","[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ..."
891,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.3890407345198038, 0.3937033345643633, 0.409...","[0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, ..."


In [29]:
predictions, responses = get_question_level_prediction(df_results)
rounded_prediction = [round(p) for p in predictions]
print(f"number of predicted questions: {len(predictions)}") 
print(f"auc: {roc_auc_score(responses, predictions)}")
print(f"accuracy: {accuracy_score(responses, rounded_prediction)}")

number of predicted questions: 84246
auc: 0.5951277881293446
accuracy: 0.5921824181563516
